# PySpark Quick Start

In [3]:
# import necessary libraries
import pandas as pd 
import numpy
import matplotlib.pyplot as plt 
from pyspark.sql import SparkSession

# create sparksession
spark = SparkSession \
    .builder \
    .appName("Pysparkexample") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [5]:
text_file = spark.read.text("example.txt")
text_file.count()

1

In [6]:
text_file.first()

Row(value='Damara Astiningtyas')